In [1]:
import os
import sys
import pandas as pd
import re
import yaml
import time

from unidecode import unidecode
from bs4 import BeautifulSoup
from langchain_text_splitters import RecursiveCharacterTextSplitter

sys.path.append('../python')

import text_tools as tt

with open('../../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

DOCID = "lamunicipalcode"
FILENAME = f"../../raw_data/{DOCID}.html"
CHUNK_SIZE = config['CHUNK_SIZE']
CHUNK_OVERLAP = config['CHUNK_OVERLAP']


In [2]:
data = pd.read_pickle(f"../../intermediate_data/{DOCID}_main.pkl")

t0 = time.time()

new_data = []
curr_chapter_id = ''
curr_article_id = ''
curr_section_id = ''
curr_start_id = data.index[0]
curr_end_id = data.index[0]
curr_text = ''
i = 0
for idx, row in data.iterrows():
    stop = (row['item_type'] in ['chapter_title', 'article_title', 'section_title']) or (len(curr_text)>2000)
    if stop:
        new_row = {
            'id': DOCID + f'_chunk_{i}',
            'doc_id': DOCID,
            'start_id': curr_start_id, 
            'end_id': curr_end_id,
            'chapter_id': curr_chapter_id, 
            'article_id': curr_article_id,
            'section_id': curr_section_id,
            'item_type': 'passage_chunk', 
            'text': curr_text
        }
        new_data.append(new_row)
        i+=1
        curr_text = ''
        if len(curr_chapter_id)>0:
            curr_text = data.loc[curr_chapter_id, 'text'] + '\n'
        if len(curr_article_id)>0:
            curr_text += ' ' + data.loc[curr_article_id, 'text'] + '\n'
        if len(curr_section_id)>0:
            curr_text += '  ' + data.loc[curr_section_id, 'text'] + '\n'
        curr_start_id = idx
        curr_end_id = idx
    
    if row['item_type'] == 'chapter_title':
        curr_chapter_id = idx
        curr_article_id = ''
        curr_section_id = ''
        curr_text = row['text'] + '\n'
        curr_start_id = idx
        curr_end_id = idx
    elif row['item_type'] == 'article_title':
        curr_article_id = idx
        curr_section_id = ''
        curr_text += ' ' + row['text'] + '\n'
        curr_start_id = idx
        curr_end_id = idx
    elif row['item_type'] == 'section_title':
        curr_section_id = idx
        curr_text += '  ' + row['text'] + '\n'
        curr_start_id = idx
        curr_end_id = idx
    else:
        curr_text += ' '*(2+row['item_level']) + row['text'] + '\n'
        curr_end_id = idx
    

t1 = time.time()
print(f"Elapsed time: {(t1-t0)/60:.2f} minutes")

new_data = pd.DataFrame.from_dict(new_data)
new_data = new_data.set_index('id')
new_data.to_csv(f"../../intermediate_data/{DOCID}_passagechunks.csv")
new_data.to_pickle(f"../../intermediate_data/{DOCID}_passagechunks.pkl")

Elapsed time: 0.04 minutes


In [7]:
print(new_data.iloc[5]['text'])

CHAPTER I GENERAL PROVISIONS AND ZONING
 GENERAL PLANNING DEPARTMENT INFORMATION
   PROCEDURES
   Requests for action by the Department of City Planning are made in the form of applications filed at the Department's two public offices. At the time an application is filed and accepted, a fee must be paid by the applicant dependent on the type of action requested and in accordance with current fee schedules established by the City Council. Cases are usually set for hearing before a Zoning Administrator, hearing officer or one of the Planning Commissions. A recommendation to a Planning Commission or a decision is rendered and distributed.
   APPEALS
   In many instances, applicants or persons aggrieved by a decision may appeal that action. On matters acted upon by the City Planning Commission, an appeal may be made to the City Council. On matters acted upon by a Zoning Administrator or the Director of Planning, an appeal may be made to Area Planning Commissions or City Planning Commission